# Project 1
# Malignant Lung Nodule Detection

In this project, you will develop a deep learning model to classify lung nodules as benign or malignant from 3D CT scans, utilizing the LUNA16 dataset. This task involves data preprocessing, model design, training, and evaluation, offering hands-on experience with medical image analysis and deep learning in PyTorch.

## 1. Create Annotation Data
As the first step, we will need to load the annotation data from Kaggle's data page: [Luna 16 Lung Cancer Dataset on Kaggle](https://www.kaggle.com/datasets/fanbyprinciple/luna-lung-cancer-dataset)

1.1 Download the annotation dataset from Kaggle.

In [ ]:
# Install Kaggle API
# ! allows us to use console commands here.
! pip install -q kaggle

from google.colab import files
files.upload()

In [ ]:
# Make a directory named kaggle and copy kaggle.json file there.
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
# Display the content of this folder
! ls ~/.kaggle

kaggle.json


In [ ]:
# Change the permissions of the file.
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the data
! kaggle datasets download -d 'fanbyprinciple/luna-lung-cancer-dataset'

Dataset URL: https://www.kaggle.com/datasets/fanbyprinciple/luna-lung-cancer-dataset
License(s): CC-BY-SA-3.0


In [ ]:
# Use unzip command to unzip the data
! unzip luna-lung-cancer-dataset.zip -d luna16

1.2 Load the `candidates_V2.csv` file as a data frame. Display the first 5 rows.

In [ ]:
import pandas as pd
candidates = pd.read_csv('luna16/candidates_V2/candidates_V2.csv')
candidates.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


1.3 Display the number of class 0 (benign) records and the number of class 1 (malignant) records. Your results should indicate that the two classes are highly imbalanced.

In [ ]:
candidates["class"].value_counts()

,count
class,
0,753418
1,1557


## 2. Find Nodule Locations
In the annotation dataset, the center of each identified lung nodule is marked with its 3D coordinates. We need to convert these coordinates into three indices to identify the specific subarray in each CT scan tensor that corresponds to the nodule.

Please follow the steps outlined in the LUNA16DataPreparation notebook to generate a CSV file named `candidates_processed.csv`, which will store the indices for the center of each lung nodule.

2.1 Load the `subset0.zip` from Google Drive using the file ID '1OFa8UhDvCrcTj1VkFLa7RjifEqMD4TAa'. Extract the zip file to reveal the .mhd and .raw files.

In [ ]:
# Install SimpleITK to handle CT scan files
! pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 11.4 MB/s eta 0:00:00


In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SimpleITK as sitk

In [ ]:
candidates = pd.read_csv('luna16/candidates_V2/candidates_V2.csv')
annotations = pd.read_csv("luna16/annotations.csv")

In [ ]:
candidates.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


In [ ]:
annotations.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1OFa8UhDvCrcTj1VkFLa7RjifEqMD4TAa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('subset0.zip')

In [ ]:
! unzip -q subset0.zip

2.2 Create a Pandas data frame that contains these 8 colums: `seriesuid`, `coordX`, `coordY`, `coordZ`, `class`, `index`, `row`, `col`. The last three columns should be calculated using the three coordiantes and the information about the origin and spacing of the corresponding CT scan.

In [ ]:
# Get list of available CT scans
all_mhd = glob.glob('subset0/*.mhd')
ids = [f[f.find('1.') : f.find('.mhd')] for f in all_mhd]

In [ ]:
# Initialize metadata dataframe
ct_info = pd.DataFrame(index=ids, columns=['originX', 'originY', 'originZ', 'spacingX', 'spacingY', 'spacingZ'])

In [ ]:
# Extract origin and spacing from each scan
for uid in ids:
    mhd_path = f'subset0/{uid}.mhd'
    image = sitk.ReadImage(mhd_path)
    ct_info.loc[uid, ['originX', 'originY', 'originZ']] = image.GetOrigin()
    ct_info.loc[uid, ['spacingX', 'spacingY', 'spacingZ']] = image.GetSpacing()

In [ ]:
# Use only candidates with available scans
candidates_small = candidates[candidates['seriesuid'].isin(ids)].copy()

In [ ]:
# Define default direction matrix (identity)
direction = np.eye(3)

In [ ]:
# Convert coordinates to voxel indices
index_list, row_list, col_list = [], [], []
for idx in candidates_small.index:
    uid = candidates_small.at[idx, 'seriesuid']
    origin = ct_info.loc[uid, ['originX', 'originY', 'originZ']].astype(float).values
    spacing = ct_info.loc[uid, ['spacingX', 'spacingY', 'spacingZ']].astype(float).values
    world_coord = candidates_small.loc[idx, ['coordX', 'coordY', 'coordZ']].values.astype(float)

    voxel = np.round(((world_coord - origin) @ np.linalg.inv(direction)) / spacing).astype(int)
    col, row, index = voxel[0], voxel[1], voxel[2]

    index_list.append(index)
    row_list.append(row)
    col_list.append(col)

In [ ]:
# Add new columns to the DataFrame
candidates_small['index'] = index_list
candidates_small['row'] = row_list
candidates_small['col'] = col_list

In [ ]:
candidates_small.head()

,seriesuid,coordX,coordY,coordZ,class,index,row,col
11673,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,-66.383107,57.143607,-60.885862,0,110,331,173
11674,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,-132.856859,23.813034,-274.350845,0,24,287,86
11675,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,84.770000,66.170000,-249.880000,0,34,343,371
11676,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,134.805360,29.696241,-268.116009,0,27,295,437
11677,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,-87.092676,41.722383,-208.831394,0,51,311,146


2.3 Save the data frame as a CSV file named `candiadates_processed.csv`. This allows you to skip Step 2.2 in future calculations.




In [ ]:
# Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Mount Google Drive as a folder in this environment
!ls drive/MyDrive

In [ ]:
# Save the index data to a CSV file
candidates_small.to_csv('drive/MyDrive/candidates_processed.csv', index=False)

In [ ]:
temp = pd.read_csv('drive/MyDrive/candidates_processed.csv')
temp.head()

,seriesuid,coordX,coordY,coordZ,class,index,row,col
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,-66.383107,57.143607,-60.885862,0,110,331,173
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,-132.856859,23.813034,-274.350845,0,24,287,86
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,84.770000,66.170000,-249.880000,0,34,343,371
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,134.805360,29.696241,-268.116009,0,27,295,437
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,-87.092676,41.722383,-208.831394,0,51,311,146


## 3. Create Data Tensors

The neural network model we will build with PyTorch requires the data to be presented in the form of a torch tensor. The input tensor should be 4-dimensional, with the dimensions representing the nodule index, channel, row, and column, respectively.

3.1 Write a double for-loop to extract the CT scan data for **the first 5,000*** nodules:
- The outer for loop goes through all the distince seriesuid's.
- For each iteration of the outer loop, load the corresponding CT-scan file and create a torch tensor to represent the scan.
- Create an inner-loop that goes through the nodules corresponding to the seriesuid:
    - Load the (index, row, col) tuple of this nodule from the data frame.
    - Extract a 32x48x48 chunk centered at the (index, row, col). If the nodule is near the edge of the image and there is not enough indices to extract, please pad with zeros to keep the overall shape unchanged.
    - Use a 4D tensor to contain all the 32x48x48 chunks. The first dimension of the 4D tensor is the index of nodule.

You may modify the above procedure as you like. Make sure that you are able to obtain a 4D tensor that contains all nodule data. **Display the shape of the 4D tensor.** The shape of the tensor should be (5000, 32, 48, 48).

**Remark** Due to the memory limit, it is impossible to load all nodule images into simultanously. Therefore, the number of nodules required in this section is reduced to 5,000. Feel free to adjust this number to prevent the out-of-memory error.

In [ ]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/candidates_processed.csv")

In [ ]:
import torch
import numpy as np
import SimpleITK as sitk

# Number of nodules to load
num_nodules = 5000

# Initialize 4D tensor and labels
chunks = torch.zeros((num_nodules, 32, 48, 48))
labels = torch.zeros(num_nodules)

# Get unique CT scan IDs
unique_seriesuid = data['seriesuid'].unique()
unique_ids = unique_seriesuid.tolist()
print("Number of unique scans:", len(unique_ids))

count = 0

for id in unique_ids:
    mhd_path = "subset0/" + id + ".mhd"

    # Load and clip CT scan
    ct_mhd = sitk.ReadImage(mhd_path)
    ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    ct_a = ct_a.clip(-1000, 1000)

    # Extract nodules from this scan
    nodules = data[data['seriesuid'] == id]
    print(f"Scan ID: {id}, nodules: {nodules.shape[0]}")

    for nodule_index in nodules.index:
        if count == num_nodules:
            break

        index, row, col = nodules.loc[nodule_index, ["index", "row", "col"]].astype(int)

        # Pad if near edges
        ct_chunk = torch.zeros([32, 48, 48])

        z_start, z_end = max(index - 16, 0), min(index + 16, ct_a.shape[0])
        y_start, y_end = max(row - 24, 0), min(row + 24, ct_a.shape[1])
        x_start, x_end = max(col - 24, 0), min(col + 24, ct_a.shape[2])

        chunk = ct_a[z_start:z_end, y_start:y_end, x_start:x_end]
        chunk = torch.tensor(chunk)

        # Determine target shape to avoid overflow
        dz, dy, dx = chunk.shape
        ct_chunk[:dz, :dy, :dx] = chunk

        chunks[count] = ct_chunk
        labels[count] = nodules.loc[nodule_index, 'class']

        count += 1

    if count == num_nodules:
        break

print("Final tensor shape:", chunks.shape)
print("Label shape:", labels.shape)



Number of unique scans: 89
Scan ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260, nodules: 707
Scan ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492, nodules: 1291
Scan ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059, nodules: 804
Scan ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987, nodules: 730
Scan ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896761494371822656720, nodules: 821
Scan ID: 1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565, nodules: 1333
Final tensor shape: torch.Size([5000, 32, 48, 48])
Label shape: torch.Size([5000])


3.2 Create a 1D tensor to contain all the class information.

In [ ]:
print("Label tensor shape:", labels.shape)
print("Sample labels:", labels[:10])

Label tensor shape: torch.Size([5000])
Sample labels: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


3.3 Split the 4D tensor into a training set and a test set. Display their shapes.

In [ ]:
from sklearn.model_selection import train_test_split
chunks_train, chunks_test, labels_train, labels_test = train_test_split(chunks, labels, test_size=0.2)
print(chunks_train.shape, labels_train.shape)
print(chunks_test.shape, labels_test.shape)

torch.Size([4000, 32, 48, 48]) torch.Size([4000])
torch.Size([1000, 32, 48, 48]) torch.Size([1000])


## 4. Model Design and Implementation

4.1 Design a neural network model with only a flatten layer and two dense layers for classifying lung nodules. You may experiment with different sizes for the hidden layers to improve the training results.

In [ ]:
number_input_values = 32 * 48 * 48
print(number_input_values)

73728


In [ ]:
import torch.nn as nn
import torch.optim as optim
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(number_input_values, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 2),
    nn.LogSoftmax(dim=1)
)

In [ ]:
learning_rate = 1e-6

4.2 Create an object to represent the loss function.

In [ ]:
loss_fn = nn.NLLLoss()

4.3 Create an object to represent the optimizer.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

4.4 Create a function to represent the training loop.


In [ ]:
# Turn the dataset into a dataloader
dataset = torch.utils.data.TensorDataset(chunks_train, labels_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                           shuffle=True)

In [ ]:
def train_model(model,train_loader,loss_fn,optimizer,n_epochs):
  for epoch in range(n_epochs):
    for img, label in train_loader:
      out = model(img)
      loss = loss_fn(out, label.long()) # label.long() represents the 0/1 labels as long integers

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

4.5 Execute the training loop. Display the change of training loss during the training process. Choose a reasonable value for the number of training epochs based on your observations.

In [ ]:
n_epochs = 50

train_model(model, train_loader, loss_fn, optimizer, n_epochs)

Epoch: 0, Loss: 0.000000
Epoch: 1, Loss: 0.000000
Epoch: 2, Loss: 0.000000
Epoch: 3, Loss: 0.000000
Epoch: 4, Loss: 0.000000
Epoch: 5, Loss: 0.000000
Epoch: 6, Loss: 0.000000
Epoch: 7, Loss: 0.000107
Epoch: 8, Loss: 0.000021
Epoch: 9, Loss: 0.003168
Epoch: 10, Loss: 0.000000
Epoch: 11, Loss: 0.798131
Epoch: 12, Loss: 0.000000
Epoch: 13, Loss: 0.000000
Epoch: 14, Loss: 0.000000
Epoch: 15, Loss: 0.000000
Epoch: 16, Loss: 0.000000
Epoch: 17, Loss: 0.000011
Epoch: 18, Loss: 0.000000
Epoch: 19, Loss: 0.000000
Epoch: 20, Loss: 0.000000
Epoch: 21, Loss: 0.000000
Epoch: 22, Loss: 0.000000
Epoch: 23, Loss: 0.000000
Epoch: 24, Loss: 0.000000
Epoch: 25, Loss: 0.000000
Epoch: 26, Loss: 0.000000
Epoch: 27, Loss: 0.000000
Epoch: 28, Loss: 0.000000
Epoch: 29, Loss: 0.000000
Epoch: 30, Loss: 0.000000
Epoch: 31, Loss: 0.000000
Epoch: 32, Loss: 0.000000
Epoch: 33, Loss: 0.000000
Epoch: 34, Loss: 0.000000
Epoch: 35, Loss: 0.000000
Epoch: 36, Loss: 0.000000
Epoch: 37, Loss: 0.000000
Epoch: 38, Loss: 0.000

## 5. Model Evaluation and Analysis

5.1 Obtain the model's prediction on the test set.

In [ ]:
# Create a data loader for the test set.
dataset = torch.utils.data.TensorDataset(chunks_test, labels_test)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                           shuffle=True)

In [ ]:
# Apply the model to the test set.
probabilities = model(chunks_test)
probabilities

tensor([[   0.0000, -261.6715],
        [   0.0000, -317.3645],
        [   0.0000, -109.4861],
        ...,
        [   0.0000, -393.2058],
        [   0.0000, -213.1809],
        [   0.0000, -211.7251]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
# Turn the probabilites into class predictions
_, predictions = torch.max(probabilities, dim=1)

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score(labels_test, predictions)
print("Accuracy:", acc)

Accuracy: 0.997


In [ ]:
# Turn the probabilites into class predictions
_, predictions = torch.max(probabilities, dim=1)
predictions[:25]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

In [ ]:
# Calculate the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(labels_test, predictions)
print(confusion_matrix)

[[997   0]
 [  3   0]]


**Confusion Matrix**

|                |Predicted Class:|     0                  |              1    |
|----------------|----------------|------------------------|-------------------|
|True Class:     |
|     0          |                |  995 (True Negatives)  | 4 (False Positive)|
|     1          |                |  1 (False Negative)    | 0 (True Positives)|

5.2 Calculate the report the following metrics:
- accuracy
- precision
- recall

In [ ]:
true_neg = confusion_matrix[0][0]
false_neg = confusion_matrix[1][0]
false_pos = confusion_matrix[0][1]
true_pos = confusion_matrix[1][1]

In [ ]:
accuracy = float((true_pos + true_neg)/1000)
print("Accuracy:", accuracy)


Accuracy: 0.997


In [ ]:
precision = float(true_pos / (true_pos + false_pos))
print("Precision:", precision)


Precision: nan


<ipython-input-81-65915a312bc6>:1: RuntimeWarning: invalid value encountered in scalar divide
  precision = float(true_pos / (true_pos + false_pos))


In [ ]:
# Recall: the number of true positives divided by the number of positive instances.
recall = float(true_pos / (true_pos + false_neg))
print("Recall:", recall)

Recall: 0.0


5.3: Discuss the model's performance.

This indicates that the model is biased toward predicting class 0 (the majority class) and completely ignores class 1 (the minority/positive class), likely because it’s underrepresented in the training data.

## 6. Data Augmentation and Retraining

To enhance the model's performance, it is essential to increase the number of malignant instances. Apply random shifting and rotation to generate new training instances, ensuring an equal number of instances in each class within the training set.

6.1 Augment the number of malignent instances in the training set.

In [ ]:
# Create a new dataset with equal numbers of positives and negatives.
num_negatives = 2500

negative_chunks = torch.zeros([num_negatives, 32, 48, 48])
negative_labels = torch.zeros([num_negatives])

# Write Your Code Here

all_files = glob.glob('subset0/*.mhd')

count = 0

for file in all_files:

    # Load the file as an array
    ct_mhd = sitk.ReadImage(file)
    ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    ct_a = ct_a.clip(-1000, 1000, ct_a)

    # Extract the ID from the file name.
    start_index = file.find('1.')
    end_index = file.find('.mhd')
    id = file[start_index:end_index]

    # for each CT scan load negative nodules from it.
    negative_nodules = data[(data['seriesuid'].str.contains(id)) & (data['class'] == 0)]

    # Loop through these nodules and create a subarray for each one.
    # We no longer need to reload the CT scan file.
    for nodule_index in negative_nodules.index:
        # Extract the indices for the center
        index, row, col = data.loc[nodule_index, ["index", "row", "col"]]

        # Build a 32x48x48 tensor to represent the nodule.
        # We need to add 0s if the nodule is too close to the boundary
        ct_chunk = torch.zeros([32, 48, 48])
        chunk = ct_a[(index-16):(index+16), (row-24):(row+24), (col-24):(col+24)]
        chunk_indices, chunk_rows, chunk_cols = chunk.shape
        ct_chunk[:chunk_indices, :chunk_rows, :chunk_cols] = torch.tensor(chunk)

        # Add this 3D tensor to the 4D tensor.
        negative_chunks[count, :, :, :] = ct_chunk

        # Add the corresponding label to the label tensor
        negative_labels[count] = 0

        count += 1
        print(count)
        if count == num_negatives - 1:
            break # break out of the inner loop
    if count == num_negatives - 1:
        break # break out of the outer loop



In [ ]:
from torchvision.transforms.v2 import RandomAffine

In [ ]:
num_positives = 100
num_copies = 25

positive_chunks = torch.zeros([num_positives * num_copies, 32, 48, 48])
positive_labels = torch.zeros([num_positives * num_copies])

# Write Your Code Here
shift_fn = RandomAffine(degrees=45, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10)

count = 0

for file in all_files:

    # Load the file as an array
    ct_mhd = sitk.ReadImage(file)
    ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    ct_a = ct_a.clip(-1000, 1000, ct_a)

    # Extract the ID from the file name.
    start_index = file.find('1.')
    end_index = file.find('.mhd')
    id = file[start_index:end_index]

    # for each CT scan load negative nodules from it.
    positive_nodules = data[(data['seriesuid'].str.contains(id)) & (data['class'] == 1)]

    # Loop through these nodules and create a subarray for each one.
    # We no longer need to reload the CT scan file.
    for nodule_index in positive_nodules.index:
        # Extract the indices for the center
        index, row, col = data.loc[nodule_index, ["index", "row", "col"]]

        # Build a 32x48x48 tensor to represent the nodule.
        # We need to add 0s if the nodule is too close to the boundary
        ct_chunk = torch.zeros([32, 48, 48])
        chunk = ct_a[(index-16):(index+16), (row-24):(row+24), (col-24):(col+24)]
        chunk_indices, chunk_rows, chunk_cols = chunk.shape
        ct_chunk[:chunk_indices, :chunk_rows, :chunk_cols] = torch.tensor(chunk)

        # Add this 3D tensor to the 4D tensor.
        positive_chunks[count, :, :, :] = ct_chunk

        # Add the corresponding label to the label tensor
        positive_labels[count] = 1

        count += 1
        print(count)

        # Augmentation: Add 24 copies of the nodule after random affine modifications
        for j in range(1, 25):
            shifted_chunk = shift_fn(ct_chunk)
            positive_chunks[num_positives * j + count, :, :, :] = shifted_chunk
            positive_labels[num_positives * j + count] = 1

        if count == num_positives - 1:
            break # break out of the inner loop
    if count == num_positives - 1:
        break # break out of the outer loop


In [ ]:
# Combine the two sets of instances
chunks = torch.cat([negative_chunks, positive_chunks], dim=0)
labels = torch.hstack([negative_labels, positive_labels])
print(chunks.shape, labels.shape)

torch.Size([5000, 32, 48, 48]) torch.Size([5000])


6.2 Retrain the neural network model on the new training set.

In [ ]:
chunks_train, chunks_test, labels_train, labels_test = train_test_split(chunks, labels, test_size=0.2)
print(chunks_train.shape, labels_train.shape)
print(chunks_test.shape, labels_test.shape)

# Turn the dataset into a dataloader
dataset = torch.utils.data.TensorDataset(chunks_train, labels_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                           shuffle=True)


torch.Size([4000, 32, 48, 48]) torch.Size([4000])
torch.Size([1000, 32, 48, 48]) torch.Size([1000])


In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(number_input_values, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 2),
    nn.LogSoftmax(dim=1)
)

In [ ]:
learning_rate = 1e-6

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

In [ ]:
n_epochs = 50

train_model(model, train_loader, loss_fn, optimizer, n_epochs)

Epoch: 0, Loss: 1.849690
Epoch: 1, Loss: 2.251556
Epoch: 2, Loss: 0.030897
Epoch: 3, Loss: 0.556668
Epoch: 4, Loss: 0.174528
Epoch: 5, Loss: 0.652966
Epoch: 6, Loss: 0.849090
Epoch: 7, Loss: 0.772426
Epoch: 8, Loss: 1.016802
Epoch: 9, Loss: 0.110779
Epoch: 10, Loss: 0.019888
Epoch: 11, Loss: 0.320961
Epoch: 12, Loss: 0.259650
Epoch: 13, Loss: 0.663508
Epoch: 14, Loss: 0.162602
Epoch: 15, Loss: 0.011629
Epoch: 16, Loss: 0.039213
Epoch: 17, Loss: 0.061623
Epoch: 18, Loss: 0.225138
Epoch: 19, Loss: 0.000361
Epoch: 20, Loss: 0.032934
Epoch: 21, Loss: 0.244257
Epoch: 22, Loss: 0.023134
Epoch: 23, Loss: 0.006970
Epoch: 24, Loss: 0.323834
Epoch: 25, Loss: 0.316080
Epoch: 26, Loss: 0.130402
Epoch: 27, Loss: 0.222948
Epoch: 28, Loss: 0.051627
Epoch: 29, Loss: 0.015068
Epoch: 30, Loss: 0.130884
Epoch: 31, Loss: 0.000101
Epoch: 32, Loss: 0.078912
Epoch: 33, Loss: 0.019224
Epoch: 34, Loss: 0.049621
Epoch: 35, Loss: 0.010971
Epoch: 36, Loss: 0.025630
Epoch: 37, Loss: 0.022442
Epoch: 38, Loss: 0.022

6.3 Perform model evaluation and compare the performance of the new model to the old model.

In [ ]:
# Create a data loader for the test set.
dataset = torch.utils.data.TensorDataset(chunks_test, labels_test)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                           shuffle=True)

In [ ]:
# Apply the model to the test set.
probabilities = model(chunks_test)
probabilities

tensor([[-8.3643e-01, -5.6785e-01],
        [-3.4571e-06, -1.2565e+01],
        [-2.5062e+01,  0.0000e+00],
        ...,
        [-7.0348e-01, -6.8292e-01],
        [-4.3806e+01,  0.0000e+00],
        [-1.3542e+01, -1.3113e-06]], grad_fn=<LogSoftmaxBackward0>)

In [ ]:
# Turn the probabilites into class predictions
_, predictions = torch.max(probabilities, dim=1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix_2 = confusion_matrix(labels_test, predictions)
print(confusion_matrix_2)

[[452  55]
 [ 40 453]]


In [ ]:
true_neg_2 = confusion_matrix_2[0][0]
false_neg_2 = confusion_matrix_2[1][0]
false_pos_2 = confusion_matrix_2[0][1]
true_pos_2 = confusion_matrix_2[1][1]

In [ ]:
accuracy_2 = float((true_pos_2 + true_neg_2)/1000)
print("Accuracy:", accuracy_2)

Accuracy: 0.905


In [ ]:
precision_2 = float(true_pos_2 / (true_pos_2 + false_pos_2))
print("Precision:", precision_2)

Precision: 0.8917322834645669


In [ ]:
recall_2 = float(true_pos_2 / (true_pos_2 + false_neg_2 ))
print("Recall:", recall_2)

Recall: 0.9188640973630832
